We reproduce Section 5.3 SimuQ data in Table 2 of our paper in this notebook, which contains the cut difference of QAOA on IBM and IonQ devices.

Note that for the convenience of the reviewer, the exhibited code below will generate the simulator results (for free). This case study is specifically designed to showcase the advantage of using interaction-based gates on real devices, which cannot be observed from classical simulation.

If you want to reproduce the real device results, set the on_simulator variable to False. CAUTION: real device experiments may cost your money to run.

The IBM provider requires you to have an IBM account to access classical simulator (see README.md). You will be given a API token from the IBM account, and you may use the commented line to create a provider, although you may not have access to the ibmq_gradalupe device. As the time when this document is written, you may access ibmq_brisbane through the open plan of IBM. 

If you don't have an IBM account, you can remove the IBM provider and the following executions related to IBM. This merely leads to the lack of IBM (ideal) curve in the first figure.

In [1]:
import numpy as np

from simuq import QSystem, Qubit
from simuq.braket import BraketProvider
from simuq.ibm import IBMProvider

bp = BraketProvider()
ibm = IBMProvider(from_file="../../../qiskit_APIKEY", hub="ibm-q-ornl", group="ornl", project="phy147")
#ibm = IBMProvider(api_key="YOUR_API_TOKEN", hub="ibm-q", group="open", project="main")

In [2]:
def GenQS(n=12, p=3):
    qs = QSystem()
    ql = [Qubit(qs) for i in range(n)]
    link = [(i, (i + 1) % n) for i in range(n)]
    if p==3:    
        parameter_list = np.array(
        [
            0.5702193 * 2,
            -0.58631086,
            0.85160685 * 2,
            -1.7058538,
            0.29468536 * 2,
            -1.132814,
        ]
    )/2
    if p==2:
        parameter_list = np.array([-3.8034,1.2438, -1.2467,-2.4899])/2
    if p==1:
        parameter_list = np.array([-0.7854,-2.3562])/2

    for i in range(p):
        h = 0
        for q0, q1 in link:
            h += parameter_list[2 * i] * ql[q0].Z * ql[q1].Z
        qs.add_evolution(h, 1)
        h = 0
        for q0 in range(n):
            h += parameter_list[2 * i+1] * ql[q0].X
        qs.add_evolution(h, 1)
    return qs

def calc_cut(count):
    cut=0
    n_shots=sum(count.values())
    for key in count:
        cut+=count[key]*calc_weight(key)/n_shots
    return cut

def calc_weight(key):
    cut=0
    for i in range(len(key)):
        if key[i]!= key[(i+1)%len(key)]:
            cut+=1
    return cut

ideal_cut_size = [9, 10, 10.5]

In [5]:
on_simulator = True

from simuq.braket import BraketIonQCircuit
state_prep = BraketIonQCircuit(12)
for i in range(12):
    state_prep.gpi2(i, np.pi / 2)
    
from qiskit import QuantumCircuit
all_h_circ = QuantumCircuit(12)
for i in range(12):
    all_h_circ.h(i)

for p in [1, 2, 3]:
    qs = GenQS(12, p)
    
    bp.compile(qs, provider="ionq", device="Aria-1", aais="heisenberg", trotter_num=4, state_prep=state_prep, verbose=-1)
    bp.run(shots=1024, on_simulator=on_simulator, verbose=-1)
    result = bp.results()
    cs = calc_cut(result)
    print(f"For p={p}, IonQ gives cut size {cs}, different from ideal value {ideal_cut_size[p-1]} by {abs(ideal_cut_size[p-1]-cs)}.")
    
    ibm.compile(qs, backend="ibmq_guadalupe", trotter_num=4, use_pulse=False, state_prep=all_h_circ, verbose=-1)
    ibm.run(on_simulator=on_simulator, verbose=-1)
    result = ibm.results(on_simulator=on_simulator)
    cs = calc_cut(result)
    print(f"For p={p}, IBM gives cut size {cs}, different from ideal value {ideal_cut_size[p-1]} by {abs(ideal_cut_size[p-1]-cs)}.")

For p=1, IonQ gives cut size 8.998046875, different from ideal value 9 by 0.001953125.
For p=1, IBM gives cut size 8.990234375, different from ideal value 9 by 0.009765625.
For p=2, IonQ gives cut size 9.990234375, different from ideal value 10 by 0.009765625.
For p=2, IBM gives cut size 10.00732421875, different from ideal value 10 by 0.00732421875.
For p=3, IonQ gives cut size 10.455078125, different from ideal value 10.5 by 0.044921875.
For p=3, IBM gives cut size 10.5126953125, different from ideal value 10.5 by 0.0126953125.
